<b>This notebook was written in August 2016 to demonstrate how to incorporate custom variability models into the CatSim treatment of AGNs (or, really, any variable sources).  It assumes version 2.3.0 of the `lsst_sims` stack.</b>

First, open an ssh tunnel to the CatSim database hosted by the University of Washington. Open a terminal window and type

`ssh -L 51433:fatboy.phys.washington.edu:1433 simsuser@gateway.astro.washington.edu`

There is some configuration that you will have to do to make sure this works. Instructions are here:

https://confluence.lsstcorp.org/display/SIM/Accessing+the+UW+CATSIM+Database


Now, we need to download a database of OpSim-simulated pointings from

https://www.lsst.org/scientists/simulations/opsim/opsim-surveys-data-directory

and specify its location with the opsimdb variable


In [ ]:
import numpy as np
import os

In [ ]:
opsimdb = os.path.join("/Users","danielsf","physics")
opsimdb = os.path.join(opsimdb, "lsst_150412", "Development", "garage")
opsimdb = os.path.join(opsimdb, "OpSimData", "kraken_1042_sqlite.db")

Light curves are generated using a class called `LightCurveGenerator`.  Sub-classes of `LightCurveGenerator` are already implemented for variable stars, type Ia supernovae, and AGNs.  Below we show how to use the `AgnLightCurveGenerator` to generate light curves using the default CatSim AGN variability model (a damped random walk).

<b>Note:</b> Throughout this notebook, we will be using the `GalaxyObj` class to establish our connection to fatboy (the UW-hosted CatSim database).  `GalaxyObj` only contains objects in the -2.5<RA<2.5, -2.25<Dec<2.25 range.  To get objects everywhere on the sky, we should be using `GalaxyTileObj`, which tiles the contents of `GalaxyObj` all over the sky. Unfortunately, this tiling makes it impossible for us to put a limit on the number of light curves returned from `GalaxyTileObj`, which is why we use `GalaxyObj`.  In the next `lsst_sims` release, this bug will be fixed, and we will be able to use `GalaxyTileObj` and only ask for a limited number of light curves.

The cell below will take about 3 minutes to run, since it has to walk through the damped random walk from `t0` to the end of your light curve.

In [ ]:
from lsst.sims.catUtils.utils import AgnLightCurveGenerator
from lsst.sims.catUtils.baseCatalogModels import GalaxyObj
import time

agn_db = GalaxyObj()

# we must tell the light curve generator about both the database of sources (fatboy)
# and the database of opsim pointings (opsimdb)
lc_gen = AgnLightCurveGenerator(agn_db, opsimdb)

ptngs = lc_gen.get_pointings((-2.5, 2.5), (-2.25, 2.25))

lc_dict, truth_dict = lc_gen.light_curves_from_pointings(ptngs, lc_per_field=10)

`lc_dict` is now a dict of light curves keyed on each AGNs `uniqueId` (a long integer uniquely identifying each object in CatSim).  `truth_dict` is also keyed on `uniqueId` and contains information characterizing the true variability parameters of each AGN (in this case, this is a json representation of a dict of parameters used by CatSim to create AGN variability; we will show you below how to customize `truth_dict` information).

Let's plot one of the light curves.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

obj_name = lc_dict.keys()[5]
lc = lc_dict[obj_name]

fig, ax = plt.subplots()

ax.errorbar(lc['r']['mjd'], lc['r']['mag'], lc['r']['error'],
            fmt='', linestyle='None')
ax.scatter(lc['r']['mjd'], lc['r']['mag'], s=5, color='r')
ax.set(xlabel='MJD', ylabel='r-band magnitude')

## Creating our own `LightCurveGenerator`

The `LightCurveGenerator` uses a CatSim `InstanceCatalog` to calculate the magnitudes and magnitude uncertainties of objects.  It expects to find these values in columns labeled `ligthCurveMag` and `sigma_lightCurveMag`.  Suppose we wanted to generate light curves of an unvarying AGN, rather than an AGN with the full CatSim variability model implemented.  To do that, we would just define an `InstanceCatalog` class which assigned the AGN's mean magnitude to `lightCurveMag` and the associated uncertainty to `sigma_lightCurveMag`.  We would then assign that class as our `LightCurveGenerator`'s `_lightCurveCatalogClass`.  See below:

<b>Note:</b> CatSim stores all of the new `InstanceCatalog` daughter classes defined in a registry. This means that re-running this or any cell that defines an `InstanceCatalog` daughter class will cause an exception to be thrown.  You will have to re-start the kernel if you wish to re-run the cell below.

In [ ]:
from lsst.sims.catalogs.decorators import compound
from lsst.sims.catUtils.mixins import PhotometryGalaxies
from lsst.sims.catUtils.utils.LightCurveGenerator import _baseLightCurveCatalog

# _lightCurveCatalogClasses must inherit from _baseLightCurveCatalog.
# _baseLightCurveCatalog defines some functionality that the LightCurveGenerator expects
class _agnMeanMagCatalog(_baseLightCurveCatalog, PhotometryGalaxies):
    
    @compound("lightCurveMag", "sigma_lightCurveMag")
    def get_lightCurvePhotometry(self):
        """
        This method calculates the lightCurveMag and sigma_lightCurveMag values expected
        by the LightCurveGenerator.  [u,g,r,i,z,y]Agn and sigma_[u,g,r,i,z,y]Agn are
        calculated by methods defined in the PhotometryGalaxies mixin imported above.
        """
        return np.array([self.column_by_name("%sAgn" % self.obs_metadata.bandpass),
                         self.column_by_name("sigma_%sAgn" % self.obs_metadata.bandpass)])
        

In [ ]:
lc_gen._lightCurveCatalogClass= _agnMeanMagCatalog

# this is just a constraint on our SQL query to make sure we do not get
# any objects that lack an AGN component
lc_gen._constraint = 'sedname_agn IS NOT NULL'

ra_bound = (-2.5, 2.5)
dec_bound = (-2.25, 2.25)
pointings = lc_gen.get_pointings(ra_bound, dec_bound, bandpass='r')

lc_dict, truth_dict = lc_gen.light_curves_from_pointings(pointings,
                                                         lc_per_field=10)


If we plot one of the light curves from this simulation, we see that it is completely flat, since we only returned the mean magnitude of the AGN.

In [ ]:
obj_name = lc_dict.keys()[5]
lc = lc_dict[obj_name]

fig, ax = plt.subplots()

ax.errorbar(lc['r']['mjd'], lc['r']['mag'], lc['r']['error'],
            fmt='', linestyle='None')
ax.scatter(lc['r']['mjd'], lc['r']['mag'], s=5, color='r')
ax.set(xlabel='MJD', ylabel='r-band magnitude')

Now let's introduce variability into our light curves.  In the `_lightCurveCatalogClass` below, we define the magnitudes `[u,g,r,i,z,y]Agn_rando` which are the mean AGN magnitudes plus random variation.  The getter method for `lightCurveMag` returns whichever of these random magnitudes is appropriate, based on the bandpass being observed at the time (encoded in the catalog's `self.obs_metadata`).

The method to calculate the uncertainty in the random magnitudes follows a simple prescription set-up in the CatSim framework.

In [ ]:
class _agnRandomMagCatalog(_baseLightCurveCatalog, PhotometryGalaxies):
    
    rng = np.random.RandomState(119)
    
    @compound('uAgn_rando', 'gAgn_rando', 'rAgn_rando',
              'iAgn_rando', 'zAgn_rando', 'yAgn_rando')
    def get_randomMagnitudes(self):
        """
        Calculate a varying magnitude that is the mean magnitude plus random noise.
        """
        n_mags = len(self.column_by_name('uAgn'))
        return np.array([self.column_by_name('uAgn') + self.rng.random_sample(n_mags)*10.0,
                         self.column_by_name('gAgn') + self.rng.random_sample(n_mags)*10.0,
                         self.column_by_name('rAgn') + self.rng.random_sample(n_mags)*10.0,
                         self.column_by_name('iAgn') + self.rng.random_sample(n_mags)*10.0,
                         self.column_by_name('zAgn') + self.rng.random_sample(n_mags)*10.0,
                         self.column_by_name('yAgn') + self.rng.random_sample(n_mags)*10.0])

    @compound('sigma_uAgn_rando', 'sigma_gAgn_rando', 'sigma_rAgn_rando',
              'sigma_iAgn_rando', 'sigma_zAgn_ranod', 'sigma_yAgn_rando')
    def get_rando_uncertainties(self):
        """
        Calculate the uncertainty in the random magnitudes.
        
        The method _magnitudeUncertaintyGetter is defined in the PhotometryGalaxies mixin.
        The arguments for that method are:
        
        list of the magnitudes for which uncertainties are to be calculated
        list of the bandpass names associated with these magnitudes
            (so that m5 in that bandpass can be looked up from self.obs_metadata)
        name of the attribute containing the bandpasses
            (self.lsstBandpassDict is set by the method that calculates [u,g,r,i,z,y]Agn)
        """
        return self._magnitudeUncertaintyGetter(['uAgn_rando', 'gAgn_rando', 'rAgn_rando',
                                                 'iAgn_rando', 'zAgn_rando', 'yAgn_rando'],
                                                ['u', 'g', 'r', 'i', 'z', 'y'],
                                                'lsstBandpassDict')
    
    @compound("lightCurveMag", "sigma_lightCurveMag")
    def get_lightCurvePhotometry(self):
        return np.array([self.column_by_name("%sAgn_rando" % self.obs_metadata.bandpass),
                         self.column_by_name("sigma_%sAgn_rando" % self.obs_metadata.bandpass)])
        
    

In [ ]:
from lsst.sims.catUtils.baseCatalogModels import GalaxyObj

lc_gen._lightCurveCatalogClass = _agnRandomMagCatalog

ra_bound = (-2.5, 2.5)
dec_bound = (-2.25, 2.25)
pointings = lc_gen.get_pointings(ra_bound, dec_bound, bandpass='r')

lc_dict_rando, truth_dict = lc_gen.light_curves_from_pointings(pointings,
                                                               lc_per_field=10)

Plotting one of these light curves shows random variation.

In [ ]:
obj_name = lc_dict_rando.keys()[5]
lc = lc_dict_rando[obj_name]

fig, ax = plt.subplots()

ax.errorbar(lc['r']['mjd'], lc['r']['mag'], lc['r']['error'],
            fmt='', linestyle='None')
ax.scatter(lc['r']['mjd'], lc['r']['mag'], s=5, color='r')
ax.set(xlabel='MJD', ylabel='r-band magnitude')

In the previous example, we used the catalog attribute `self.lsstBandpassDict` when calling `_magnitudeUncertaintyGetter`.  Ordinarily, `self.lsstBandpassDict` is set by the method which calculates the mean AGN magnitudes (the mean AGN magnitudes are calculated by loading in model SEDs and integrating them over the LSST bandpasses).  Below, we alter how the mean LSST magnitudes are calculated so that we can show how `self.lsstBandpassDict` might be set by the user.  Note that we have renamed the mean magnitudes `[u,g,r,i,z,y]Agn_x`.  This is because `[u,g,r,i,z,y]Agn` are already calculated by a method defined in the `PhotometryGalaxies` mixin, and we do not want to confuse CatSim.

In [ ]:
from lsst.sims.photUtils import BandpassDict

class _alternateAgnCatalog(_baseLightCurveCatalog, PhotometryGalaxies):

    @compound('uAgn_x', 'gAgn_x', 'rAgn_x', 'iAgn_x', 'zAgn_x', 'yAgn_x')
    def get_magnitudes(self):
        """
        Add a component to the mean magnitude that grows linearly with time
        (which probably means it is not a mean magnitude any more...)
        """
        
        if not hasattr(self, 'lsstBandpassDict'):
            self.lsstBandpassDict = BandpassDict.loadTotalBandpassesFromFiles()
        
        delta = 5.0 * (self.obs_metadata.mjd.TAI-59580.0)/3650.0

        # self._magnitudeGetter is defined in the PhotometryGalaxies mixin.
        # It's arguments are: a str indicating which galaxy component is being
        # simulated ('agn', 'disk', or 'bulge'), the BandpassDict containing
        # the bandpasses of the survey, a list of the columns defined by this
        # current getter method
        return self._magnitudeGetter('agn', self.lsstBandpassDict,
                                     self.get_magnitudes._colnames) + delta
    
    @compound('sigma_uAgn_x', 'sigma_gAgn_x', 'sigma_rAgn_x',
              'sigma_iAgn_x', 'sigma_zAgn_x', 'sigma_yAgn_x')
    def get_uncertainties(self):
        return self._magnitudeUncertaintyGetter(['uAgn_x', 'gAgn_x', 'rAgn_x',
                                                 'iAgn_x', 'zAgn_x', 'yAgn_x'],
                                                ['u', 'g', 'r', 'i', 'z', 'y'],
                                                'lsstBandpassDict')
    
    @compound("lightCurveMag", "sigma_lightCurveMag")
    def get_lightCurvePhotometry(self):
        return np.array([self.column_by_name("%sAgn_x" % self.obs_metadata.bandpass),
                         self.column_by_name("sigma_%sAgn_x" % self.obs_metadata.bandpass)])

In [ ]:
lc_gen._lightCurveCatalogClass = _alternateAgnCatalog

ra_bound = (-2.5, 2.5)
dec_bound = (-2.25, 2.25)
pointings = lc_gen.get_pointings(ra_bound, dec_bound, bandpass='r')

lc_dict_alt, truth_dict = lc_gen.light_curves_from_pointings(pointings,
                                                               lc_per_field=10)

Plotting one of these light curves shows an unintersting linear rise in magnitude.

In [ ]:
obj_name = lc_dict_alt.keys()[5]
lc = lc_dict_alt[obj_name]

fig, ax = plt.subplots()

ax.errorbar(lc['r']['mjd'], lc['r']['mag'], lc['r']['error'],
            fmt='', linestyle='None')
ax.scatter(lc['r']['mjd'], lc['r']['mag'], s=5, color='r')
ax.set(xlabel='MJD', ylabel='r-band magnitude')

Up until now we have been ignoring the existence of the `truth_dict`, which contains the reference information about the specific variability of each object in the `_lightCurveCatalogClass`'s catalog.  Just as the magnitude and uncertainty in our light curves are generated by columns known as `lightCurveMag` and `sigma_lightCurveMag`, the `truth_dict` is constructed from a column called `truthInfo`.  To calculate it, you must define a method `get_truthInfo()` in your `_lightCurveCatalogClass`.  The method should return a numpy array in which each element is the truth information for the corresponding AGN.  Below, we construct truth info that consists of a period, phase, and amplitude for a sinusoidal light curve component.  We add it to the AGN magnitudes calculated by `get_magnitudes()`.

In [ ]:
class _variableAgnCatalog(_baseLightCurveCatalog, PhotometryGalaxies):

    rng = np.random.RandomState(88)
    
    def get_truthInfo(self):
        if not hasattr(self, 'truth_cache'):
            self.truth_cache = {}
            
        # get the uniqueIds of all of the AGn
        # (mostly so you know how many of them there are)
        id_val = self.column_by_name('uniqueId')
        
        output = []
        for ii in id_val:
            if ii in self.truth_cache:
                output.append(self.truth_cache[ii])
            else:
                period = self.rng.random_sample()*365.25
                phase = self.rng.random_sample()*2.0*np.pi
                amplitude = self.rng.random_sample()*10.0
                output.append((period, phase, amplitude))

        return np.array(output)
    
    @compound('uAgn_x', 'gAgn_x', 'rAgn_x', 'iAgn_x', 'zAgn_x', 'yAgn_x')
    def get_magnitudes(self):
        
        if not hasattr(self, 'lsstBandpassDict'):
            self.lsstBandpassDict = BandpassDict.loadTotalBandpassesFromFiles()
        
        delta = 5.0 * (self.obs_metadata.mjd.TAI-59580.0)/3650.0

        var_params = self.column_by_name('truthInfo')
        wave = [vv[2]*np.sin(2.0*np.pi*self.obs_metadata.mjd.TAI/vv[0] + vv[1]) for vv in var_params]
        
        return self._magnitudeGetter('agn', self.lsstBandpassDict,
                                     self.get_magnitudes._colnames) + delta + wave
    
    @compound('sigma_uAgn_x', 'sigma_gAgn_x', 'sigma_rAgn_x',
              'sigma_iAgn_x', 'sigma_zAgn_x', 'sigma_yAgn_x')
    def get_uncertainties(self):
        return self._magnitudeUncertaintyGetter(['uAgn_x', 'gAgn_x', 'rAgn_x',
                                                 'iAgn_x', 'zAgn_x', 'yAgn_x'],
                                                ['u', 'g', 'r', 'i', 'z', 'y'],
                                                'lsstBandpassDict')
    
    @compound("lightCurveMag", "sigma_lightCurveMag")
    def get_lightCurvePhotometry(self):
        return np.array([self.column_by_name("%sAgn_x" % self.obs_metadata.bandpass),
                         self.column_by_name("sigma_%sAgn_x" % self.obs_metadata.bandpass)])

In [ ]:
lc_gen._lightCurveCatalogClass = _variableAgnCatalog

ra_bound = (-2.5, 2.5)
dec_bound = (-2.25, 2.25)
pointings = lc_gen.get_pointings(ra_bound, dec_bound, bandpass='r')

lc_dict_var, truth_dict = lc_gen.light_curves_from_pointings(pointings,
                                                               lc_per_field=10)

Plotting light curves from this `LightCurveGenerator`, we again see non-linear variation in the magnitude.

In [ ]:
obj_name = lc_dict_var.keys()[5]
lc = lc_dict_var[obj_name]

fig, ax = plt.subplots()

ax.errorbar(lc['r']['mjd'], lc['r']['mag'], lc['r']['error'],
            fmt='', linestyle='None')
ax.scatter(lc['r']['mjd'], lc['r']['mag'], s=5, color='r')
ax.set(xlabel='MJD', ylabel='r-band magnitude')

The `truth_dict` contains the three parameters characterizing the sinusoidal component of the variation.

In [ ]:
truth_dict[obj_name]